In [10]:
using Flux, Statistics, ProgressMeter, Plots

In [2]:
noisy = rand(Float32, 2, 1000)
truth = [xor(col[1]>.5, col[2]>.5) for col in eachcol(noisy)];

In [3]:
model = Chain(
    Dense(2 => 3, tanh),
    BatchNorm(3),
    Dense(3 => 2),
    softmax
) |> gpu;

┌ Info: The GPU function is being called but the GPU is not accessible. 
│ Defaulting back to the CPU. (No action is required if you want to run on the CPU).
└ @ Flux /Users/lionelpeer/.julia/packages/Flux/n3cOc/src/functor.jl:301


In [4]:
out1 = model(noisy |> gpu) |> cpu

2×1000 Matrix{Float32}:
 0.463301  0.488332  0.454565  0.505679  …  0.471615  0.454076  0.440655
 0.536699  0.511668  0.545435  0.494321     0.528385  0.545924  0.559345

In [5]:
target = Flux.onehotbatch(truth, [true, false])
loader = Flux.DataLoader((noisy, target) |> gpu, batchsize=64, shuffle=true)
optim = Flux.setup(Flux.Adam(0.01), model);

In [6]:
# losses = []
# @showprogress for epoch in 1:1_000
#     for (x,y) in loader
#         loss, grads = Flux.withgradient(model) do m
#             y_hat = m(x)
#             Flux.crossentropy(y_hat, y)
#         end
#         Flux.update!(optim, model, grads[1])
#         push!(losses, loss)
#     end
# end

In [16]:
grads = 0
loss = 0
for (i, (x, y)) in enumerate(loader)
    loss, grads = Flux.withgradient(m -> Flux.crossentropy(m(x), y), model)
    print(loss, "\n", grads)
    if i == 1
        break
    end
end

0.7316772
((layers = ((weight = Float32[-0.021530159 0.07032434; 0.047946744 0.08559638; 0.0016346322 -0.03002288], bias = Float32[0.022522835, -0.03351566, 0.00018574623], σ = nothing), (λ = nothing, β = Float32[-0.020301037, -0.041499853, 0.005920957], γ = Float32[0.024135558, 0.08530525, 0.0065081446], μ = nothing, σ² = nothing, ϵ = -4.5947294f0, momentum = nothing, affine = nothing, track_stats = nothing, active = nothing, chs = nothing), (weight = Float32[-0.05844722 -0.10105412 0.05403688; 0.058447212 0.10105412 -0.05403688], bias = Float32[0.049161427, -0.04916141], σ = nothing), nothing),),)

In [17]:
loss

0.7316772f0

In [30]:
grads[1].layers

((weight = Float32[-0.021530159 0.07032434; 0.047946744 0.08559638; 0.0016346322 -0.03002288], bias = Float32[0.022522835, -0.03351566, 0.00018574623], σ = nothing), (λ = nothing, β = Float32[-0.020301037, -0.041499853, 0.005920957], γ = Float32[0.024135558, 0.08530525, 0.0065081446], μ = nothing, σ² = nothing, ϵ = -4.5947294f0, momentum = nothing, affine = nothing, track_stats = nothing, active = nothing, chs = nothing), (weight = Float32[-0.05844722 -0.10105412 0.05403688; 0.058447212 0.10105412 -0.05403688], bias = Float32[0.049161427, -0.04916141], σ = nothing), nothing)